In [59]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import Trainer, TrainingArguments, TrainerCallback
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import tensorflow as tf
import torch
import os
from datasets import Dataset
import time
from sklearn.model_selection import train_test_split

In [2]:
# Configurar dispositivo como variable global
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def check_gpu_memory():
    allocated_memory = torch.cuda.memory_allocated()
    cached_memory = torch.cuda.memory_reserved()
    print(f"Memoria GPU asignada: {allocated_memory / 1024 ** 2:.2f} MB")
    print(f"Memoria GPU reservada: {cached_memory / 1024 ** 2:.2f} MB")
    print(f"Memoria libre: {torch.cuda.get_device_properties(0).total_memory / 1024 ** 2 - allocated_memory / 1024 ** 2:.2f} MB")

check_gpu_memory()  # Verifica antes de entrenar

Memoria GPU asignada: 0.00 MB
Memoria GPU reservada: 0.00 MB
Memoria libre: 4095.56 MB


In [4]:

def clear_gpu_memory():
    # Comando para restablecer la GPU usando nvidia-smi
    os.system('nvidia-smi --gpu-reset -i 0')

    # Limpiar la caché de memoria de CUDA
    torch.cuda.empty_cache()

    # También puedes intentar vaciar la caché de la memoria de TensorFlow, si lo estás usando
    try:
        import tensorflow as tf
        tf.keras.backend.clear_session()
        print("Sesión de TensorFlow limpiada.")
    except ImportError:
        pass

    # Verificación de memoria
    print("Verificando el estado de la memoria de la GPU después de limpiar...")
    time.sleep(2)  # Un pequeño retraso para asegurar que los cambios se apliquen
    os.system('nvidia-smi')

# Llamada a la función para limpiar la memoria de la GPU
clear_gpu_memory()


Sesión de TensorFlow limpiada.
Verificando el estado de la memoria de la GPU después de limpiar...


In [5]:
from dotenv import load_dotenv
import os
from huggingface_hub import login

load_dotenv()  # Carga variables del archivo .env
token = os.getenv("HF_TOKEN")
login(token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [39]:
"""
Carga de datos
"""
# Rutas de los archivos procesados
ruta_argumentos = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\procesed\argumentos_socraticos_procesados.csv"
ruta_dialogos = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\procesed\dialogos_socraticos_procesados.csv"
ruta_escritos = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\procesed\escritos_socraticos_procesados.csv"

# Cargar los datos
df_argumentos = pd.read_csv(ruta_argumentos)
df_dialogos = pd.read_csv(ruta_dialogos)
df_escritos = pd.read_csv(ruta_escritos)

In [45]:
# Reemplaza NAN en df_dialogos
df_dialogos['dialogo_socratico'] = df_dialogos['dialogo_socratico'].fillna('[]')
df_dialogos['preguntas_socraticas'] = df_dialogos['preguntas_socraticas'].fillna('[]')

# Reemplaza NAN en df_escritos
df_escritos['fragmentos_socraticos'] = df_escritos['fragmentos_socraticos'].fillna('[]')
df_escritos['preguntas_filosoficas'] = df_escritos['preguntas_filosoficas'].fillna('[]')

# Reemplaza NAN en df_argumentos
df_argumentos['estructura_argumentativa'] = df_argumentos['estructura_argumentativa'].fillna('[]')
df_argumentos['posibles_contradicciones'] = df_argumentos['posibles_contradicciones'].fillna('[]')
df_argumentos['fragmentos_socraticos'] = df_argumentos['fragmentos_socraticos'].fillna('[]')

In [46]:
df_argumentos.head(10)

,titulo,argumento_procesado,estructura_argumentativa,posibles_contradicciones,fragmentos_socraticos
0,Argumento de Clitofon,clitofon acusado sócrates censurar conversació...,"{'premisas': [], 'conclusiones': []}",[],[]
1,Argumento de Lisis,objeto diálogo amistad título lleno esperanza ...,{'premisas': ['La filosofía quiere más precisi...,"['Sin embargo, el hombre que ama a otro puede ...","[',¿Es el amigo el que ama o el que es amado?'..."
2,Argumento de Timeo de Locres,timeo nacido locrio epicefiriano grecia cuyo n...,"{'premisas': [], 'conclusiones': []}",[],"['Por otra parte, ¿cómo puede admitirse esta i..."
3,Argumento de Fedro,tradición necesidad discutir obra juventud pla...,{'premisas': ['Un tratado de Aristóteles prese...,"[',Sin embargo, existe un lazo entre estas dos...","[',¿Cuál es el objeto del diálogo?', ',En el ,..."
4,Argumento de segundo Alcibíades,orar él ligeramente dirigir dios súplica pedir...,"{'premisas': [], 'conclusiones': []}",[],['Lo mejor es fiarse a los dioses mismos para ...
5,Argumento de Ion,sócrates calle atenas rapsodista ion efeso ven...,{'premisas': ['Cada uno de estos tiene un géne...,[],"['Pues entonces, ¿qué es?', '¿Y las conoce Ion..."
6,Argumento de Eutidemo,composición perfectamente estudiado cuyo págin...,{'premisas': ['Jamás quizá el heredero de Sócr...,"['Pero Sócrates, abandonando de repente su pa...","['Pero, ¿qué prueba el interrogatorio en favor..."
7,Argumento de Laques,objeto valor ocasión valor educación hijo cien...,"{'premisas': [], 'conclusiones': ['Es así que ...",[],"['La verdadera cuestión que debe resolverse, s..."
8,Argumento de Filebo,primero página aparecer sentado diverso soluci...,{'premisas': ['Si es cosa que haya de entrar o...,[],['Desde las primeras páginas aparece sentado e...
9,Argumento de Fedón,precedente diálogo mero serie pregunta respues...,{'premisas': ['Y en cuanto a acortar el términ...,[],"['El , no es, como los precedentes diálogos, u..."


In [47]:
# Eliminar 'Argumento de ' de la columna 'titulo'
df_argumentos['titulo'] = df_argumentos['titulo'].str.replace('Argumento de ', '', regex=False)

# Mostrar el resultado
print(df_argumentos)

                             titulo  \
0                          Clitofon   
1                             Lisis   
2                   Timeo de Locres   
3                             Fedro   
4                segundo Alcibíades   
5                               Ion   
6                          Eutidemo   
7                            Laques   
8                            Filebo   
9                             Fedón   
10                      El banquete   
11                         Carmides   
12  Argumento del primer Alcibiades   
13                           Critón   
14             Apología de Sócrates   
15                         Eutifrón   

                                  argumento_procesado  \
0   clitofon acusado sócrates censurar conversació...   
1   objeto diálogo amistad título lleno esperanza ...   
2   timeo nacido locrio epicefiriano grecia cuyo n...   
3   tradición necesidad discutir obra juventud pla...   
4   orar él ligeramente dirigir dios súplica pedir.

In [48]:
df_dialogos.head(10)

,titulo,dialogo_procesado,preguntas_socraticas,dialogo_socratico
0,Clitofon,clitofon acusado sócrates censurar conversac...,[],[]
1,Fedón,relato histórico consistir pintura sensible ...,"['¿No debe el filósofo desear morir?', '¿Tiene...",[]
2,Timeo de Locres,timeo nacido locrio epicefiriano grecia cuyo...,[],[]
3,segundo Alcibíades,orar él ligeramente dirigir dios súplica ped...,[],[]
4,Clitofon,sócrates clitofon hijo aristonimo instante c...,"['Me ha sucedido muchas veces, Sócrates, que e...","Sócrates Clitofon, hijo de Aristonimo, me han..."
5,Timeo de Locres,timeo locres causa inteligencia causa intenc...,[],[]
6,Eutidemo,composición perfectamente estudiado conversa...,"['Pero, ¿qué prueba el interrogatorio en favor...",[]
7,El banquete,objeto diálogo amor preámbulo cuyo circunsta...,['¡No es esto una imagen sensible de su frugal...,[]
8,El segundo Alcibíades,sócrates alcibíades ir orar templo alcibíade...,"['Sócrates Alcibíades, ¿vas a orar en este tem...","Sócrates Alcibíades, ¿vas a orar en este temp..."
9,Filebo,primero página aparecer sentado sócrates sen...,"['¿Qué hombre se considera dichoso, aun en med...",[]


In [49]:
df_escritos.head(10)

,titulo,contenido_procesado,preguntas_filosoficas,fragmentos_socraticos
0,Clitofon,clitofon acusado sócrat censurar conversación ...,[],[]
1,Cartas,correspondencia comprender trece carta dirigid...,[],La correspondencia que sigue comprende trece c...
2,De lo justo,justo comienzo asunto diálogo injusto ignoranc...,"['¿qué es lo justo?', ',¿qué demonio ha inspir...","¿Qué es lo justo? Este es el comienzo , y el a..."
3,De la virtud,hé serie pregunta respuesta interés literario ...,['hé aquí otra serie de preguntas y respuestas...,Hé aquí otra serie de preguntas y respuestas d...
4,Axioco,vida viaje natural necesario término término m...,"['morir es hacerse insensible, es decir, incap...","La vida no es más que un viaje; y es natural, ..."
5,Erixias,superior diálogo inmediatamente precedente mér...,[],[]
6,Minos,diálogo dejar analogía interlocutor sócrat ami...,[],[]
7,Epinomis o el filósofo,complemento conclusión ley edición impreso peq...,[],[]
8,Timeo de Locres,timeo nacido locrio epicefiriano grecia cuyo n...,"['por otra parte, ¿cómo puede admitirse esta i...","Timeo, nacido entre los locrios epicefirianos ..."
9,Los rivales,platón autor cualquiera pequeño diálogo propon...,['¿consiste el ser filósofo en querer conocer ...,"Platón, o el autor, cualquiera que él sea, de ..."


In [50]:
print("Columnas en df_argumentos:", df_argumentos.columns.tolist())
print("Columnas en df_dialogos:", df_dialogos.columns.tolist())
print("Columnas en df_escritos:", df_escritos.columns.tolist())

Columnas en df_argumentos: ['titulo', 'argumento_procesado', 'estructura_argumentativa', 'posibles_contradicciones', 'fragmentos_socraticos']
Columnas en df_dialogos: ['titulo', 'dialogo_procesado', 'preguntas_socraticas', 'dialogo_socratico']
Columnas en df_escritos: ['titulo', 'contenido_procesado', 'preguntas_filosoficas', 'fragmentos_socraticos']


In [51]:
# Combinar df_argumentos y df_dialogos por la columna 'titulo'
df_combinado = pd.merge(df_argumentos, df_dialogos, on='titulo', how='outer')

In [52]:
# Luego combinar el resultado con df_escritos por la columna 'titulo'
df_combinado = pd.merge(df_combinado, df_escritos, on='titulo', how='outer')

In [53]:
# Mostrar el DataFrame combinado
print(df_combinado)

                             titulo  \
0              Apología de Sócrates   
1              Apología de Sócrates   
2   Argumento del primer Alcibiades   
3                            Axioco   
4                          Carmides   
..                              ...   
73                  Timeo de Locres   
74                  Timeo de Locres   
75         Timeo o de la naturaleza   
76                primer Alcibiades   
77               segundo Alcibíades   

                                  argumento_procesado  \
0   apología dividir él parte objeto ,en preceder ...   
1   apología dividir él parte objeto ,en preceder ...   
2   naturaleza humano conocimiento considerado pun...   
3                                                 NaN   
4   complicado diálogo marchar llanamente objeto s...   
..                                                ...   
73  timeo nacido locrio epicefiriano grecia cuyo n...   
74  timeo nacido locrio epicefiriano grecia cuyo n...   
75                

In [54]:
# Reemplaza NAN en df_dialogos
df_combinado['argumento_procesado'] = df_combinado['argumento_procesado'].fillna('[]')
df_combinado['estructura_argumentativa'] = df_combinado['estructura_argumentativa'].fillna('[]')
df_combinado['posibles_contradicciones'] = df_combinado['posibles_contradicciones'].fillna('[]')
df_combinado['fragmentos_socraticos_x'] = df_combinado['fragmentos_socraticos_x'].fillna('[]')
df_combinado['dialogo_procesado'] = df_combinado['dialogo_procesado'].fillna('[]')
df_combinado['preguntas_socraticas'] = df_combinado['preguntas_socraticas'].fillna('[]')
df_combinado['dialogo_socratico'] = df_combinado['dialogo_socratico'].fillna('[]')
df_combinado['contenido_procesado'] = df_combinado['contenido_procesado'].fillna('[]')
df_combinado['preguntas_filosoficas'] = df_combinado['preguntas_filosoficas'].fillna('[]')
df_combinado['fragmentos_socraticos_y'] = df_combinado['fragmentos_socraticos_y'].fillna('[]')

In [57]:
df_combinado.head(20)

,titulo,argumento_procesado,estructura_argumentativa,posibles_contradicciones,fragmentos_socraticos_x,dialogo_procesado,preguntas_socraticas,dialogo_socratico,contenido_procesado,preguntas_filosoficas,fragmentos_socraticos_y
0,Apología de Sócrates,"apología dividir él parte objeto ,en preceder ...",{'premisas': ['Estos dos cargos se llamaban y ...,"['Así es que contra sus intereses más caros, s...","['¿Era este el camino de congraciarse, teniend...",apología dividir él parte objeto preceder de...,"['¿Era este el camino de congraciarse, teniend...",[],"apología dividir él parte objeto ,en preceder ...","['¿era este el camino de congraciarse, teniend...","La apología puede dividirse en tres partes, ca..."
1,Apología de Sócrates,"apología dividir él parte objeto ,en preceder ...",{'premisas': ['Estos dos cargos se llamaban y ...,"['Así es que contra sus intereses más caros, s...","['¿Era este el camino de congraciarse, teniend...",ateniense impresión discurso acusador confes...,"['¿Qué decían mis primeros acusadores?', 'He o...",[],"apología dividir él parte objeto ,en preceder ...","['¿era este el camino de congraciarse, teniend...","La apología puede dividirse en tres partes, ca..."
2,Argumento del primer Alcibiades,naturaleza humano conocimiento considerado pun...,{'premisas': ['Porque para aprenderlo por sí m...,[],['Lo justo y lo útil son por su orden objeto d...,[],[],[],[],[],[]
3,Axioco,[],[],[],[],[],[],[],vida viaje natural necesario término término m...,"['morir es hacerse insensible, es decir, incap...","La vida no es más que un viaje; y es natural, ..."
4,Carmides,complicado diálogo marchar llanamente objeto s...,"{'premisas': ['—No, dice Sócrates, porque la s...",[],"['¿Qué es por lo tanto la sabiduría?,La mesura...",complicado diálogo marchar llanamente objeto...,"['¿Qué es por lo tanto la sabiduría?', 'Por lo...",[],[],[],[]
5,Carmides,complicado diálogo marchar llanamente objeto s...,"{'premisas': ['—No, dice Sócrates, porque la s...",[],"['¿Qué es por lo tanto la sabiduría?,La mesura...",sócrates haber llegar víspera llegada ejérci...,['Entré en la palestra de Taureas —¡Oh Sócrate...,Sócrates Habiendo llegado la víspera de la ll...,[],[],[]
6,Carmides o de la sabiduría,[],[],[],[],[],[],[],complicado diálogo marchar llanamente objeto s...,"['¿qué es por lo tanto la sabiduría?,la mesura...",Nada menos complicado que este diálogo. Marcha...
7,Cartas,[],[],[],[],[],[],[],correspondencia comprender trece carta dirigid...,[],La correspondencia que sigue comprende trece c...
8,Clitofon,clitofon acusado sócrates censurar conversació...,"{'premisas': [], 'conclusiones': []}",[],[],clitofon acusado sócrates censurar conversac...,[],[],clitofon acusado sócrat censurar conversación ...,[],[]
9,Clitofon,clitofon acusado sócrates censurar conversació...,"{'premisas': [], 'conclusiones': []}",[],[],sócrates clitofon hijo aristonimo instante c...,"['Me ha sucedido muchas veces, Sócrates, que e...","Sócrates Clitofon, hijo de Aristonimo, me han...",clitofon acusado sócrat censurar conversación ...,[],[]


In [58]:
# Preparar modelo y tokenizador
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.to(device)


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [25]:
# Función de tokenización
def tokenize_texts(texts):
    return tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=512,
        return_tensors="pt"
    )

In [60]:
# Preparar datasets
texts = df_combinado.tolist()
train_texts, val_texts = train_test_split(texts, test_size=0.1, random_state=42)

train_encodings = tokenize_texts(train_texts)
val_encodings = tokenize_texts(val_texts)

train_dataset = Dataset.from_dict(train_encodings)
val_dataset = Dataset.from_dict(val_encodings)

AttributeError: 'DataFrame' object has no attribute 'tolist'

In [14]:
"""
Configuracion de Entrenamiento
"""

# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir='Z:/results',  # Ruta a tu disco duro externo para los checkpoints
    per_device_train_batch_size=4,  # Tamaño de lote para entrenamiento
    per_device_eval_batch_size=4,   # Tamaño de lote para evaluación
    num_train_epochs=3,             # Número de épocas de entrenamiento
    logging_dir='Z:/logs',         # Ruta para los registros en el disco duro externo
    logging_steps=10,               # Pasos para imprimir el log
    save_steps=100,                 # Cada cuántos pasos se guarda el checkpoint
    save_total_limit=2,             # Número máximo de checkpoints guardados
    fp16=True,                      # Habilitar precisión mixta para optimizar el uso de memoria
    gradient_accumulation_steps=4,
    evaluation_strategy="steps",
    eval_steps=100,
)



In [ ]:
# Habilitar gradient checkpointing
model.gradient_checkpointing_enable()

In [ ]:
# Inicializar trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)



In [ ]:
# Entrenar modelo
trainer.train()

In [ ]:
# Guardar modelo
save_path = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\models"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

In [ ]:
# Ruta para guardar el modelo entrenado
save_path = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\models"

# Guardar el modelo y el tokenizador
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Modelo y tokenizador guardados en: {save_path}")

In [14]:
"""
Evaluación rapida del modelo
"""
# Generar texto de prueba
input_text = "Sócrates dijo:"
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

# Generar texto con el modelo entrenado
output = model.generate(input_ids, max_length=150, num_return_sequences=1, no_repeat_ngram_size=2, early_stopping=True)

# Decodificar y mostrar el resultado
print(tokenizer.decode(output[0], skip_special_tokens=True))

''